In [1]:
import os
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pickle
import random
from scipy import sparse
import itertools
from scipy.io import savemat, loadmat
import re
import string
import numpy as np
# import data
import pandas as pd
import numpy as np

In [11]:
#Lenta = pd.read_csv('/home/skoltsov/koltcov_sergei/data/lenta/lenta_lem_1.csv', sep=';', header=None, encoding='utf8').iloc[:,0:2]
#Lenta = Lenta.rename(columns={0:'Text', 1:'n_words'})

#---------------------------------------------------------------
Lenta = pd.read_csv('C:/koltcov_python_scripts/Word_embeding project/text_preprocessing/WOS/level2/WoS11967_lem_clear_2.csv', sep=';', header=None, encoding='utf8').iloc[:,0:2]
Lenta = Lenta.rename(columns={0:'Text', 1:'n_words'})

In [12]:
corpus = list(Lenta.Text)

init_docs = [re.findall(r'''[\w']+|[.,!?;-~{}`´_<=>:/@*()&'$%#"]''', corpus[doc])
             for doc in range(len(corpus))] #выделяем слова

def contains_punctuation(w):
    return any(char in string.punctuation for char in w)

def contains_numeric(w):
    return any(char.isdigit() for char in w)

#убираем большие буквы, цифры, однобуквенные слова и соединяем обратно
init_docs = [[w.lower() for w in init_docs[doc] if not contains_punctuation(w)]
             for doc in range(len(init_docs))]
init_docs = [[w for w in init_docs[doc] if not contains_numeric(w)] for doc in range(len(init_docs))]
init_docs = [[w for w in init_docs[doc] if len(w)>1] for doc in range(len(init_docs))]
init_docs = [" ".join(init_docs[doc]) for doc in range(len(init_docs))]


# Обычный частотный векторайзер
print('counting document frequency of words...')
cvectorizer = CountVectorizer(min_df=10, max_df=0.7, stop_words=None)
cvz = cvectorizer.fit_transform(init_docs).sign()

# Создаем словарь
print('building the vocabulary...')
sum_counts = cvz.sum(axis=0)
v_size = sum_counts.shape[1]
sum_counts_np = np.zeros(v_size, dtype=int) ###здесь просто распаковываем лист
for v in range(v_size):########################
    sum_counts_np[v] = sum_counts[0,v]#########
word2id = dict([(w, cvectorizer.vocabulary_.get(w)) for w in cvectorizer.vocabulary_])
id2word = dict([(cvectorizer.vocabulary_.get(w), w) for w in cvectorizer.vocabulary_])
del cvectorizer
print('  initial vocabulary size: {}'.format(v_size))

#Сортируем словарь
idx_sort = np.argsort(sum_counts_np)
vocab_aux = [id2word[idx_sort[cc]] for cc in range(v_size)]

# Фильтруем stop words,если есть (можно фильтровать на этапе векторайзера)
#vocab_aux = [w for w in vocab_aux if w not in stops]
#print('  vocabulary size after removing stopwords from list: {}'.format(len(vocab_aux)))

# Create dictionary and inverse dictionary
vocab = vocab_aux
del vocab_aux
word2id = dict([(w, j) for j, w in enumerate(vocab)])
id2word = dict([(j, w) for j, w in enumerate(vocab)])

# Split in train/test/valid
print('tokenizing documents and splitting into train/test/valid...')
num_docs_tr = len(corpus)-500
trSize = num_docs_tr-100
tsSize = 500
vaSize = 100
idx_permute = np.random.permutation(num_docs_tr).astype(int)

#for i in os.listdir('lenta_embeddings'):        
 #   with open('lenta_embeddings/' + i, 'rb') as f:
  #      vocab_2 = []
   #     for l in f:
    #        line = l.decode('ANSI').split()
     #       word = line[0]
      #      vocab_2.append(word)
       # print(len(vocab_2))
        #vocab=list(set(vocab) & set(vocab_2))


# Удаляем слова, которых нет в тренировочном датасете
vocab = list(set([w for idx_d in range(trSize) for w in init_docs[idx_permute[idx_d]].split()
                  if w in word2id]))


word2id = dict([(w, j) for j, w in enumerate(vocab)])
id2word = dict([(j, w) for j, w in enumerate(vocab)])
print('Окончательный размер словаря: {}'.format(len(vocab)))

# Split in train/test/valid + меняем слова на id
docs_tr = [[word2id[w] for w in init_docs[idx_permute[idx_d]].split() 
            if w in word2id] for idx_d in range(trSize)]
docs_va = [[word2id[w] for w in init_docs[idx_permute[idx_d+trSize]].split()
            if w in word2id] for idx_d in range(vaSize)]
docs_ts = [[word2id[w] for w in init_docs[idx_d+num_docs_tr].split() 
            if w in word2id] for idx_d in range(tsSize)]

print('  number of documents (train): {} [this should be equal to {}]'.format(len(docs_tr), trSize))
print('  number of documents (test): {} [this should be equal to {}]'.format(len(docs_ts), tsSize))
print('  number of documents (valid): {} [this should be equal to {}]'.format(len(docs_va), vaSize))

# Удаляем пустые документы
print('removing empty documents...')

def remove_empty(in_docs):
    return [doc for doc in in_docs if doc!=[]]

docs_tr = remove_empty(docs_tr)
docs_ts = remove_empty(docs_ts)
docs_va = remove_empty(docs_va)

# Удаляем тестовые дкументы длины = 1
docs_ts = [doc for doc in docs_ts if len(doc)>1]

# Разделяем тестовую выборку на две части
print('splitting test documents in 2 halves...')
docs_ts_h1 = [[w for i,w in enumerate(doc) if i<=len(doc)/2.0-1] for doc in docs_ts]
docs_ts_h2 = [[w for i,w in enumerate(doc) if i>len(doc)/2.0-1] for doc in docs_ts]

print('creating lists of words...')

#Создаем списки со словами из списков с документами
def create_list_words(in_docs):
    return [x for y in in_docs for x in y]

words_tr = create_list_words(docs_tr)
words_ts = create_list_words(docs_ts)
words_ts_h1 = create_list_words(docs_ts_h1)
words_ts_h2 = create_list_words(docs_ts_h2)
words_va = create_list_words(docs_va)

print('  len(words_tr): ', len(words_tr))
print('  len(words_ts): ', len(words_ts))
print('  len(words_ts_h1): ', len(words_ts_h1))
print('  len(words_ts_h2): ', len(words_ts_h2))
print('  len(words_va): ', len(words_va))

# Получаем индексы документов для списка сл словами
print('getting doc indices...')

def create_doc_indices(in_docs):
    aux = [[j for i in range(len(doc))] for j, doc in enumerate(in_docs)]
    return [int(x) for y in aux for x in y]

doc_indices_tr = create_doc_indices(docs_tr)
doc_indices_ts = create_doc_indices(docs_ts)
doc_indices_ts_h1 = create_doc_indices(docs_ts_h1)
doc_indices_ts_h2 = create_doc_indices(docs_ts_h2)
doc_indices_va = create_doc_indices(docs_va)

print('  len(np.unique(doc_indices_tr)): {} [this should be {}]'.format(len(np.unique(doc_indices_tr)), len(docs_tr)))
print('  len(np.unique(doc_indices_ts)): {} [this should be {}]'.format(len(np.unique(doc_indices_ts)), len(docs_ts)))
print('  len(np.unique(doc_indices_ts_h1)): {} [this should be {}]'.format(len(np.unique(doc_indices_ts_h1)), len(docs_ts_h1)))
print('  len(np.unique(doc_indices_ts_h2)): {} [this should be {}]'.format(len(np.unique(doc_indices_ts_h2)), len(docs_ts_h2)))
print('  len(np.unique(doc_indices_va)): {} [this should be {}]'.format(len(np.unique(doc_indices_va)), len(docs_va)))

# Number of documents in each set
n_docs_tr = len(docs_tr)
n_docs_ts = len(docs_ts)
n_docs_ts_h1 = len(docs_ts_h1)
n_docs_ts_h2 = len(docs_ts_h2)
n_docs_va = len(docs_va)

# Remove unused variables
del docs_tr
del docs_ts
del docs_ts_h1
del docs_ts_h2
del docs_va

# Create bow representation
print('creating bow representation...')

def create_bow(doc_indices, words, n_docs, vocab_size):
    return sparse.coo_matrix(([1]*len(doc_indices),(doc_indices, words)), 
                             shape=(n_docs, vocab_size)).tocsr()

bow_tr = create_bow(doc_indices_tr, words_tr, n_docs_tr, len(vocab))
bow_ts = create_bow(doc_indices_ts, words_ts, n_docs_ts, len(vocab))
bow_ts_h1 = create_bow(doc_indices_ts_h1, words_ts_h1, n_docs_ts_h1, len(vocab))
bow_ts_h2 = create_bow(doc_indices_ts_h2, words_ts_h2, n_docs_ts_h2, len(vocab))
bow_va = create_bow(doc_indices_va, words_va, n_docs_va, len(vocab))

del words_tr
del words_ts
del words_ts_h1
del words_ts_h2
del words_va
del doc_indices_tr
del doc_indices_ts
del doc_indices_ts_h1
del doc_indices_ts_h2
del doc_indices_va

# Write the vocabulary to a file
path_save = 'C:/koltcov_python_scripts/Word_embeding project/text_preprocessing/WOS/level2/'

#print(vocab)

with open(path_save + 'vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)
del vocab

# Split bow intro token/value pairs
print('splitting bow intro token/value pairs and saving to disk...')

def split_bow(bow_in, n_docs):
    indices = [[w for w in bow_in[doc,:].indices] for doc in range(n_docs)]
    counts = [[c for c in bow_in[doc,:].data] for doc in range(n_docs)]
    return indices, counts

bow_tr_tokens, bow_tr_counts = split_bow(bow_tr, n_docs_tr)
savemat(path_save + 'bow_tr_tokens.mat', {'tokens': bow_tr_tokens}, do_compression=True)
savemat(path_save + 'bow_tr_counts.mat', {'counts': bow_tr_counts}, do_compression=True)
del bow_tr
del bow_tr_tokens
del bow_tr_counts

bow_ts_tokens, bow_ts_counts = split_bow(bow_ts, n_docs_ts)
savemat(path_save + 'bow_ts_tokens.mat', {'tokens': bow_ts_tokens}, do_compression=True)
savemat(path_save + 'bow_ts_counts.mat', {'counts': bow_ts_counts}, do_compression=True)
del bow_ts
del bow_ts_tokens
del bow_ts_counts

bow_ts_h1_tokens, bow_ts_h1_counts = split_bow(bow_ts_h1, n_docs_ts_h1)
savemat(path_save + 'bow_ts_h1_tokens.mat', {'tokens': bow_ts_h1_tokens}, do_compression=True)
savemat(path_save + 'bow_ts_h1_counts.mat', {'counts': bow_ts_h1_counts}, do_compression=True)
del bow_ts_h1
del bow_ts_h1_tokens
del bow_ts_h1_counts

bow_ts_h2_tokens, bow_ts_h2_counts = split_bow(bow_ts_h2, n_docs_ts_h2)
savemat(path_save + 'bow_ts_h2_tokens.mat', {'tokens': bow_ts_h2_tokens}, do_compression=True)
savemat(path_save + 'bow_ts_h2_counts.mat', {'counts': bow_ts_h2_counts}, do_compression=True)
del bow_ts_h2
del bow_ts_h2_tokens
del bow_ts_h2_counts

bow_va_tokens, bow_va_counts = split_bow(bow_va, n_docs_va)
savemat(path_save + 'bow_va_tokens.mat', {'tokens': bow_va_tokens}, do_compression=True)
savemat(path_save + 'bow_va_counts.mat', {'counts': bow_va_counts}, do_compression=True)
del bow_va
del bow_va_tokens
del bow_va_counts

print('Data ready !!')
print('*************')

counting document frequency of words...
building the vocabulary...
  initial vocabulary size: 5581
tokenizing documents and splitting into train/test/valid...
Окончательный размер словаря: 5581
  number of documents (train): 11367 [this should be equal to 11367]
  number of documents (test): 500 [this should be equal to 500]
  number of documents (valid): 100 [this should be equal to 100]
removing empty documents...
splitting test documents in 2 halves...
creating lists of words...
  len(words_tr):  1201648
  len(words_ts):  53813
  len(words_ts_h1):  26780
  len(words_ts_h2):  27033
  len(words_va):  10497
getting doc indices...
  len(np.unique(doc_indices_tr)): 11367 [this should be 11367]
  len(np.unique(doc_indices_ts)): 500 [this should be 500]
  len(np.unique(doc_indices_ts_h1)): 500 [this should be 500]
  len(np.unique(doc_indices_ts_h2)): 500 [this should be 500]
  len(np.unique(doc_indices_va)): 100 [this should be 100]
creating bow representation...
splitting bow intro token/